<a href="https://colab.research.google.com/github/junsookim1/Everday_Practice/blob/master/Koyang_Bike_Data(%EC%9B%94_%EB%8B%A8%EC%9C%84_%EA%B5%AC%EB%B6%84_%EC%9D%B4%EB%8F%99%EA%B1%B0%EB%A6%AC%2C%EC%8B%9C%EA%B0%84)_%2B_%EC%9E%A1%EB%8B%A4%ED%95%9C_%EA%B2%83%EB%93%A4_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import missingno as msno
from haversine import haversine

In [ ]:
df=pd.read_csv('01.운영이력_ver1(한글컬럼, 이용거리).csv')

/home/jun/anaconda3/envs/jun/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
del df['Unnamed: 0']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142688 entries, 0 to 3142687
Data columns (total 14 columns):
 #   Column      Dtype  
---  ------      -----  
 0   대여 번호       int64  
 1   대여 시간       object 
 2   대여 스테이션 번호  int64  
 3   대여 거치대 번호   int64  
 4   반납 시간       object 
 5   반납 스테이션 번호  int64  
 6   반납 거치대 번호   int64  
 7   추정 이동 거리    float64
 8   회원 구분       object 
 9   회원 번호       int64  
 10  비회원 번호      float64
 11  자전거 번호      object 
 12  관제 반납       float64
 13  이용 시간(s)    float64
dtypes: float64(4), int64(6), object(4)
memory usage: 335.7+ MB


In [ ]:
df['대여 시간'] =  pd.to_datetime(df['대여 시간'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df['반납 시간'] =  pd.to_datetime(df['반납 시간'], format='%Y-%m-%d %H:%M:%S', errors='raise')

In [ ]:
df.set_index('대여 시간',inplace=True)

In [ ]:
df

,대여 번호,대여 스테이션 번호,대여 거치대 번호,반납 시간,반납 스테이션 번호,반납 거치대 번호,추정 이동 거리,회원 구분,회원 번호,비회원 번호,자전거 번호,관제 반납,이용 시간(s)
대여 시간,,,,,,,,,,,,,
2017-01-01 00:00:41,15945541,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,791.0
2017-01-01 00:01:03,15945542,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,2961.0
2017-01-01 00:01:50,15945543,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,3600.0
2017-01-01 00:02:09,15945544,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,829.0
2017-01-01 00:03:32,15945545,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,912.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 23:52:39,21718608,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN,377.0
2019-12-31 23:53:59,21718610,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN,240.0
2019-12-31 23:55:27,21718611,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN,657.0


In [ ]:
df.resample(rule='M')

In [ ]:
df_time_mean = df.resample(rule='M').mean()
df_time_mean

,대여 번호,대여 스테이션 번호,대여 거치대 번호,반납 스테이션 번호,반납 거치대 번호,추정 이동 거리,회원 번호,비회원 번호,관제 반납,이용 시간(s)
대여 시간,,,,,,,,,,
2017-01-31,1.597914e+07,251.525084,13.030678,224.960557,11.360890,8903.751314,119062.867273,6.583283e+05,1.085021,1496.286333
2017-02-28,1.604964e+07,251.205081,13.016073,226.804139,11.476540,8764.307791,111347.085157,7.173678e+05,1.088579,1566.171357
2017-03-31,1.614859e+07,250.220995,13.165041,226.798195,11.701454,9163.443077,109030.985564,8.694015e+05,1.136145,1602.969166
2017-04-30,1.631283e+07,248.978421,13.247433,226.674350,11.836205,9258.841761,104351.429379,1.077659e+06,1.163123,1846.604920
2017-05-31,1.653766e+07,248.597575,13.169264,227.547131,11.696698,8744.256897,113643.179918,9.449728e+05,1.204122,1696.929611
2017-06-30,1.679187e+07,247.378849,13.218650,226.173022,11.707605,8933.243546,123115.481696,8.360385e+05,1.199439,1607.333249
2017-07-31,1.702057e+07,248.954300,13.276504,226.489580,11.606204,8494.810944,128799.904320,7.388355e+05,1.178824,1520.677718
2017-08-31,1.723413e+07,248.472131,13.327906,226.602776,11.685162,7345.921713,125747.898998,8.144060e+05,1.171409,1575.344774
2017-09-30,1.748207e+07,247.557130,13.333497,226.740946,11.752574,6057.658008,123853.487194,8.894366e+05,1.199583,1627.275733


In [ ]:
df_time_mean = df_time_mean.loc[:,['추정 이동 거리','이용 시간(s)']]
df_time_mean  = df_time_mean.loc[:,['추정 이동 거리','이용 시간(s)']]
df_time_mean

,추정 이동 거리,이용 시간(s)
대여 시간,,
2017-01-31,8903.751314,1496.286333
2017-02-28,8764.307791,1566.171357
2017-03-31,9163.443077,1602.969166
2017-04-30,9258.841761,1846.604920
2017-05-31,8744.256897,1696.929611
2017-06-30,8933.243546,1607.333249
2017-07-31,8494.810944,1520.677718
2017-08-31,7345.921713,1575.344774
2017-09-30,6057.658008,1627.275733


In [ ]:
df_time_sum = df.resample(rule='M').sum()
df_time_sum = df_time_sum.loc[:,['추정 이동 거리','이용 시간(s)']]
df_time_sum

,추정 이동 거리,이용 시간(s)
대여 시간,,
2017-01-31,2.930848e+08,54969071.0
2017-02-28,2.807909e+08,55542701.0
2017-03-31,5.813746e+08,111927322.0
2017-04-30,9.356615e+08,204688769.0
2017-05-31,1.130536e+09,240186811.0
2017-06-30,1.217074e+09,240037540.0
2017-07-31,8.896700e+08,175493812.0
2017-08-31,8.213255e+08,193606722.0
2017-09-30,8.544145e+08,251339246.0


In [ ]:
df_1 = pd.merge(left = df_time_mean, right = df_time_sum, left_index = True, right_index = True, how = "inner")
df_1.columns = ['추정 이동 거리(평균)','이용 시간(s)(평균)','추정 이동 거리(합)','이용 시간(s)(합)']
df_1

,추정 이동 거리(평균),이용 시간(s)(평균),추정 이동 거리(합),이용 시간(s)(합)
대여 시간,,,,
2017-01-31,8903.751314,1496.286333,2.930848e+08,54969071.0
2017-02-28,8764.307791,1566.171357,2.807909e+08,55542701.0
2017-03-31,9163.443077,1602.969166,5.813746e+08,111927322.0
2017-04-30,9258.841761,1846.604920,9.356615e+08,204688769.0
2017-05-31,8744.256897,1696.929611,1.130536e+09,240186811.0
2017-06-30,8933.243546,1607.333249,1.217074e+09,240037540.0
2017-07-31,8494.810944,1520.677718,8.896700e+08,175493812.0
2017-08-31,7345.921713,1575.344774,8.213255e+08,193606722.0
2017-09-30,6057.658008,1627.275733,8.544145e+08,251339246.0


In [ ]:
df_1.to_csv('월별 이동거리_시간(평균,합).csv') # 저장

In [ ]:
df_1 = pd.read_csv('월별 이동거리_시간(평균,합).csv')# 불러오기

## 월별 거치대 이용 현황

In [ ]:
df_1=pd.read_csv('01.운영이력_ver1(한글컬럼, 이용거리).csv')

/home/jun/anaconda3/envs/jun/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
del df_1['Unnamed: 0']

In [ ]:
df_1

,대여 번호,대여 시간,대여 스테이션 번호,대여 거치대 번호,반납 시간,반납 스테이션 번호,반납 거치대 번호,추정 이동 거리,회원 구분,회원 번호,비회원 번호,자전거 번호,관제 반납,이용 시간(s)
0,15945541,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,791.0
1,15945542,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,2961.0
2,15945543,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,3600.0
3,15945544,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,829.0
4,15945545,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,912.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142683,21718608,2019-12-31 23:52:39,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN,377.0
3142684,21718610,2019-12-31 23:53:59,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN,240.0
3142685,21718611,2019-12-31 23:55:27,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN,657.0
3142686,21718613,2019-12-31 23:59:14,264,4,2020-01-01 00:10:39,210,13,0.0,12,180671,0.0,1A84400000F804,NaN,685.0


In [ ]:
df_ren_month = []
for i in range(len(df_1['대여 시간'])):
    df_ren_month.append(df_1['대여 시간'][i][:7])
len(df_ren_month)

3142688

In [ ]:
df_return_month = []
for i in range(len(df_1['반납 시간'])):
    df_return_month.append(df_1['반납 시간'][i][:7])
len(df_return_month )

3142688

In [ ]:
df_ren_month = pd.DataFrame(df_ren_month, columns=['대여 시간(M)']) 
df_return_month = pd.DataFrame(df_return_month, columns=['반납 시간(M)']) 

In [ ]:
df_2 = pd.concat([df_1,df_ren_month,df_return_month], axis = 1)
df_2 

,대여 번호,대여 시간,대여 스테이션 번호,대여 거치대 번호,반납 시간,반납 스테이션 번호,반납 거치대 번호,추정 이동 거리,회원 구분,회원 번호,비회원 번호,자전거 번호,관제 반납,이용 시간(s),대여 시간(M),반납 시간(M)
0,15945541,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,791.0,2017-01,2017-01
1,15945542,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,2961.0,2017-01,2017-01
2,15945543,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,3600.0,2017-01,2017-01
3,15945544,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,829.0,2017-01,2017-01
4,15945545,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,912.0,2017-01,2017-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142683,21718608,2019-12-31 23:52:39,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN,377.0,2019-12,2019-12
3142684,21718610,2019-12-31 23:53:59,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN,240.0,2019-12,2019-12
3142685,21718611,2019-12-31 23:55:27,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN,657.0,2019-12,2020-01
3142686,21718613,2019-12-31 23:59:14,264,4,2020-01-01 00:10:39,210,13,0.0,12,180671,0.0,1A84400000F804,NaN,685.0,2019-12,2020-01


In [ ]:
df_3 = df_2.loc[:,['대여 시간(M)','대여 스테이션 번호','반납 시간(M)','반납 스테이션 번호','추정 이동 거리','이용 시간(s)','회원 번호','비회원 번호']]
df_3 # 스테이션을 얼마나 이용했는지 구하기 위해서
# df_3.to_csv('월별 스테이션 이용 횟수.csv') # csv저장

,대여 시간(M),대여 스테이션 번호,반납 시간(M),반납 스테이션 번호,추정 이동 거리,이용 시간(s),회원 번호,비회원 번호
0,2017-01,213,2017-01,260,0.0,791.0,164203,0.0
1,2017-01,231,2017-01,231,31039.0,2961.0,187551,0.0
2,2017-01,119,2017-01,0,NaN,3600.0,168994,0.0
3,2017-01,121,2017-01,133,15490.0,829.0,183971,0.0
4,2017-01,320,2017-01,259,0.0,912.0,167475,0.0
...,...,...,...,...,...,...,...,...
3142683,2019-12,222,2019-12,219,0.0,377.0,236957,0.0
3142684,2019-12,343,2019-12,311,0.0,240.0,0,4124665.0
3142685,2019-12,333,2020-01,322,0.0,657.0,0,4124667.0
3142686,2019-12,264,2020-01,210,0.0,685.0,180671,0.0


In [ ]:
# df_3.to_csv('월별 거치대 이용 현황.csv') # 저장
df_3 = pd.read_csv('월별 스테이션 이용 현황.csv')# 불러오기

In [ ]:
df_3

,Unnamed: 0,대여 시간(M),대여 스테이션 번호,반납 시간(M),반납 스테이션 번호,추정 이동 거리,이용 시간(s),회원 번호,비회원 번호
0,0,2017-01,213,2017-01,260,0.0,791.0,164203,0.0
1,1,2017-01,231,2017-01,231,31039.0,2961.0,187551,0.0
2,2,2017-01,119,2017-01,0,NaN,3600.0,168994,0.0
3,3,2017-01,121,2017-01,133,15490.0,829.0,183971,0.0
4,4,2017-01,320,2017-01,259,0.0,912.0,167475,0.0
...,...,...,...,...,...,...,...,...,...
3142683,3142683,2019-12,222,2019-12,219,0.0,377.0,236957,0.0
3142684,3142684,2019-12,343,2019-12,311,0.0,240.0,0,4124665.0
3142685,3142685,2019-12,333,2020-01,322,0.0,657.0,0,4124667.0
3142686,3142686,2019-12,264,2020-01,210,0.0,685.0,180671,0.0


In [ ]:
del df_3['Unnamed: 0']

In [ ]:
df_4 = pd.read_csv('스테이션 별 이동거리.csv')# 불러오기

In [ ]:
del df_4['Unnamed: 0']

In [ ]:
df_4.columns = ['대여 스테이션 번호','반납 스테이션 번호', 'harv이동거리']

In [ ]:
df_4

,대여 스테이션 번호,반납 스테이션 번호,harv이동거리
0,101,101,0.000000
1,101,103,810.593473
2,101,104,7749.401651
3,101,105,414.953376
4,101,106,715.798911
...,...,...,...
26891,992,350,9768.976190
26892,992,351,9278.739241
26893,992,352,8104.699800
26894,992,353,7993.663782


In [ ]:
df_5 = df_3.merge(df_4, on=['대여 스테이션 번호','반납 스테이션 번호'],how='inner') # 아닌 걸로

In [ ]:
df_5

,대여 시간(M),대여 스테이션 번호,반납 시간(M),반납 스테이션 번호,추정 이동 거리,이용 시간(s),회원 번호,비회원 번호,harv이동거리
0,2017-01,213,2017-01,260,0.0,791.0,164203,0.0,1707.564955
1,2017-01,213,2017-01,260,0.0,900.0,3781,0.0,1707.564955
2,2017-01,213,2017-01,260,0.0,1132.0,188080,0.0,1707.564955
3,2017-02,213,2017-02,260,0.0,2833.0,119748,0.0,1707.564955
4,2017-02,213,2017-02,260,0.0,731.0,188408,0.0,1707.564955
...,...,...,...,...,...,...,...,...,...
2744418,2019-12,265,2019-12,215,0.0,9358.0,222948,0.0,2662.109146
2744419,2019-12,263,2019-12,309,0.0,1904.0,236873,0.0,1648.332619
2744420,2019-12,229,2019-12,351,0.0,1835.0,0,4124197.0,3705.849781
2744421,2019-12,339,2019-12,141,0.0,3594.0,228390,0.0,11012.758702


In [ ]:
df_5[df_5['추정 이동 거리']==0]

,대여 시간(M),대여 스테이션 번호,반납 시간(M),반납 스테이션 번호,추정 이동 거리,이용 시간(s),회원 번호,비회원 번호,harv이동거리
0,2017-01,213,2017-01,260,0.0,791.0,164203,0.0,1707.564955
1,2017-01,213,2017-01,260,0.0,900.0,3781,0.0,1707.564955
2,2017-01,213,2017-01,260,0.0,1132.0,188080,0.0,1707.564955
3,2017-02,213,2017-02,260,0.0,2833.0,119748,0.0,1707.564955
4,2017-02,213,2017-02,260,0.0,731.0,188408,0.0,1707.564955
...,...,...,...,...,...,...,...,...,...
2744418,2019-12,265,2019-12,215,0.0,9358.0,222948,0.0,2662.109146
2744419,2019-12,263,2019-12,309,0.0,1904.0,236873,0.0,1648.332619
2744420,2019-12,229,2019-12,351,0.0,1835.0,0,4124197.0,3705.849781
2744421,2019-12,339,2019-12,141,0.0,3594.0,228390,0.0,11012.758702


In [ ]:
df_3[df_3['반납 스테이션 번호']==0]

,대여 시간(M),대여 스테이션 번호,반납 시간(M),반납 스테이션 번호,추정 이동 거리,이용 시간(s),회원 번호,비회원 번호
2,2017-01,119,2017-01,0,NaN,3600.0,168994,0.0
14,2017-01,234,2017-01,0,NaN,807.0,180418,0.0
18,2017-01,229,2017-01,0,NaN,3600.0,0,3185087.0
21,2017-01,306,2017-01,0,NaN,3578.0,0,3185091.0
26,2017-01,213,2017-01,0,NaN,929.0,180418,0.0
...,...,...,...,...,...,...,...,...
3142655,2019-12,121,2020-01,0,NaN,3354.0,236770,0.0
3142658,2019-12,214,2020-01,0,NaN,3323.0,182974,0.0
3142664,2019-12,343,2019-12,0,NaN,601.0,229770,0.0
3142679,2019-12,211,2020-01,0,NaN,3386.0,196111,0.0


In [ ]:
df_3[df_3['반납 스테이션 번호']==999]

,대여 시간(M),대여 스테이션 번호,반납 시간(M),반납 스테이션 번호,추정 이동 거리,이용 시간(s),회원 번호,비회원 번호
172317,2017-04,162,2017-04,999,NaN,31663.0,188356,0.0
504175,2017-06,165,2017-06,999,NaN,1154.0,194976,0.0
628713,2017-07,171,2017-07,999,NaN,818.0,196777,0.0
1338134,2018-04,162,2018-04,999,NaN,26216.0,206633,0.0
1494071,2018-05,161,2018-05,999,NaN,23982.0,205400,0.0
1745962,2018-08,163,2018-08,999,NaN,378.0,199528,0.0
1754909,2018-08,163,2018-08,999,NaN,619.0,199528,0.0
2238867,2019-03,172,2019-03,999,NaN,934.0,220675,0.0
2259389,2019-03,168,2019-03,999,NaN,132.0,219868,0.0
2488459,2019-05,172,2019-06,999,NaN,457.0,220675,0.0


In [ ]:
df_2017_01 = df_3[df_3['대여 시간(M)'] == '2017-01']
df_2017_01

,Unnamed: 0,대여 시간(M),대여 스테이션 번호,반납 시간(M),반납 스테이션 번호,추정 이동 거리,이용 시간(s),회원 번호,비회원 번호
0,0,2017-01,213,2017-01,260,0.0,791.0,164203,0.0
1,1,2017-01,231,2017-01,231,31039.0,2961.0,187551,0.0
2,2,2017-01,119,2017-01,0,NaN,3600.0,168994,0.0
3,3,2017-01,121,2017-01,133,15490.0,829.0,183971,0.0
4,4,2017-01,320,2017-01,259,0.0,912.0,167475,0.0
...,...,...,...,...,...,...,...,...,...
36732,36732,2017-01,247,2017-02,230,16465.0,734.0,181196,0.0
36733,36733,2017-01,217,2017-01,206,0.0,430.0,188276,0.0
36734,36734,2017-01,349,2017-02,305,0.0,2924.0,0,3193460.0
36735,36735,2017-01,349,2017-02,305,0.0,2890.0,0,3193461.0


In [ ]:
type(df_3['대여 스테이션 번호'].unique())

numpy.ndarray

In [ ]:
df_3['반납 스테이션 번호'].unique()

array([260, 231,   0, 133, 259, 210, 213, 305, 113, 215, 235, 234, 125,
       302, 331, 225, 143, 248, 321, 206, 313, 201, 212, 227, 236, 306,
       219, 240, 214, 258, 118, 217, 322, 229, 247, 252, 230, 237, 308,
       333, 220, 319, 204, 316, 249, 311, 342, 222, 339, 110, 141, 330,
       202, 119, 164, 203, 228, 318, 343, 111, 211, 307, 131, 246, 238,
       148, 226, 245, 323, 312, 101, 232, 349, 303, 309, 239, 137, 250,
       121, 320, 166, 221, 317, 218, 334, 257, 324, 251, 345, 326, 340,
       310, 329, 124, 233, 224, 315, 207, 328, 256, 253, 112, 169, 314,
       327, 254, 205, 350, 104, 171, 165, 114, 103, 116, 341, 129, 161,
       223, 167, 130, 140, 128, 347, 172, 105, 126, 106, 144, 304, 115,
       346, 127, 142, 123, 170, 146, 216, 163, 255, 301, 138, 162, 139,
       173, 168, 348, 992, 999, 176, 177, 178, 261, 262, 351, 265, 264,
       263, 998])

## 스테이션 별 이용 현황

In [ ]:
df_3 = pd.read_csv('월별 스테이션 이용 현황.csv')# 불러오기

In [ ]:
month_hour = df_3['대여 시간(M)'].unique().tolist()
month_hour

['2017-01',
 '2017-02',
 '2017-03',
 '2017-04',
 '2017-05',
 '2017-06',
 '2017-07',
 '2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-01',
 '2018-02',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10',
 '2018-11',
 '2018-12',
 '2019-01',
 '2019-02',
 '2019-03',
 '2019-04',
 '2019-05',
 '2019-06',
 '2019-07',
 '2019-08',
 '2019-09',
 '2019-10',
 '2019-11',
 '2019-12']

In [ ]:
month_hour_x = []
for i in month_hour:
    month_hour_x.append('M_{}'.format(i)) 
month_hour_x

['M_2017-01',
 'M_2017-02',
 'M_2017-03',
 'M_2017-04',
 'M_2017-05',
 'M_2017-06',
 'M_2017-07',
 'M_2017-08',
 'M_2017-09',
 'M_2017-10',
 'M_2017-11',
 'M_2017-12',
 'M_2018-01',
 'M_2018-02',
 'M_2018-03',
 'M_2018-04',
 'M_2018-05',
 'M_2018-06',
 'M_2018-07',
 'M_2018-08',
 'M_2018-09',
 'M_2018-10',
 'M_2018-11',
 'M_2018-12',
 'M_2019-01',
 'M_2019-02',
 'M_2019-03',
 'M_2019-04',
 'M_2019-05',
 'M_2019-06',
 'M_2019-07',
 'M_2019-08',
 'M_2019-09',
 'M_2019-10',
 'M_2019-11',
 'M_2019-12']

In [ ]:
rent_station_num = df_3['대여 스테이션 번호'].unique().tolist() # 대여 반납 같이 사용 예정
rent_station_num.sort() 
rent_station_num

[101,
 103,
 104,
 105,
 106,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 118,
 119,
 121,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 133,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 146,
 148,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 176,
 177,
 178,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 326,
 327,
 328,
 329,
 330,
 331,
 333,
 334,
 339,
 340,
 341,
 342,
 343,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 992,
 998]

In [ ]:
return_station_num = df_3['반납 스테이션 번호'].unique().tolist() # 대여 반납 같이 사용 예정
return_station_num.sort() 
return_station_num

[0,
 101,
 103,
 104,
 105,
 106,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 118,
 119,
 121,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 133,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 146,
 148,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 176,
 177,
 178,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 326,
 327,
 328,
 329,
 330,
 331,
 333,
 334,
 339,
 340,
 341,
 342,
 343,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 992,
 998,
 999]

In [ ]:
df_month_hour = pd.DataFrame(month_hour, columns=['대여 시간(M)'])
df_month_hour

,대여 시간(M)
0,2017-01
1,2017-02
2,2017-03
3,2017-04
4,2017-05
5,2017-06
6,2017-07
7,2017-08
8,2017-09
9,2017-10


In [ ]:
df_rent_station_num = pd.DataFrame(rent_station_num, columns=['대여 스테이션 번호'])
df_rent_station_num # 대여 스테이션 데이터 프레임

,대여 스테이션 번호
0,101
1,103
2,104
3,105
4,106
...,...
151,349
152,350
153,351
154,992


In [ ]:
df_return_station_num = pd.DataFrame(return_station_num, columns=['반납 스테이션 번호'])
df_return_station_num # 반납 스테이션 데이터 프레임

,반납 스테이션 번호
0,0
1,101
2,103
3,104
4,105
...,...
153,350
154,351
155,992
156,998


In [ ]:
df_rent_station_num_1 = pd.DataFrame(rent_station_num, columns=['대여 스테이션 번호'])
df_rent_station_num_1 # 대여 스테이션 데이터 프레임

,대여 스테이션 번호
0,101
1,103
2,104
3,105
4,106
...,...
151,349
152,350
153,351
154,992


In [ ]:
df_return_station_num = pd.DataFrame(return_station_num, columns=['반납 스테이션 번호'])
df_return_station_num # 반납 스테이션 데이터 프레임

,반납 스테이션 번호
0,0
1,101
2,103
3,104
4,105
...,...
153,350
154,351
155,992
156,998


## 각 월별로 데이터 보기

In [ ]:
# t = []
# for i,hour in zip(month_hour_x, month_hour):
#        t.append(df_3[df_3['대여 시간(M)'] == hour][:156].reset_index()['대여 시간(M)'])
# for문으로 돌려서 하는 방법 생각해 보기

In [ ]:
M_2017_01 = df_3[df_3['대여 시간(M)'] == '2017-01'][:158].reset_index()['대여 시간(M)']
M_2017_01

0      2017-01
1      2017-01
2      2017-01
3      2017-01
4      2017-01
        ...   
153    2017-01
154    2017-01
155    2017-01
156    2017-01
157    2017-01
Name: 대여 시간(M), Length: 158, dtype: object

In [ ]:
# 3년치 데이터 월별로 분리 / dataframe 합쳐주기
M_2017_01 = df_3[df_3['반납 시간(M)'] == '2017-01'][:158].reset_index()['반납 시간(M)']
df_2017_01= pd.concat([M_2017_01, df_return_station_num ],axis=1)
M_2017_02 = df_3[df_3['반납 시간(M)'] == '2017-02'][:158].reset_index()['반납 시간(M)']
df_2017_02= pd.concat([M_2017_02, df_return_station_num ],axis=1)
M_2017_03 = df_3[df_3['반납 시간(M)'] == '2017-03'][:158].reset_index()['반납 시간(M)']
df_2017_03= pd.concat([M_2017_03, df_return_station_num ],axis=1)
M_2017_04 = df_3[df_3['반납 시간(M)'] == '2017-04'][:158].reset_index()['반납 시간(M)']
df_2017_04= pd.concat([M_2017_04, df_return_station_num ],axis=1)
M_2017_05 = df_3[df_3['반납 시간(M)'] == '2017-05'][:158].reset_index()['반납 시간(M)']
df_2017_05= pd.concat([M_2017_05, df_return_station_num ],axis=1)
M_2017_06 = df_3[df_3['반납 시간(M)'] == '2017-06'][:158].reset_index()['반납 시간(M)']
df_2017_06= pd.concat([M_2017_06, df_return_station_num ],axis=1)
M_2017_07 = df_3[df_3['반납 시간(M)'] == '2017-07'][:158].reset_index()['반납 시간(M)']
df_2017_07= pd.concat([M_2017_07, df_return_station_num ],axis=1)
M_2017_08 = df_3[df_3['반납 시간(M)'] == '2017-08'][:158].reset_index()['반납 시간(M)']
df_2017_08= pd.concat([M_2017_08, df_return_station_num ],axis=1)
M_2017_09 = df_3[df_3['반납 시간(M)'] == '2017-09'][:158].reset_index()['반납 시간(M)']
df_2017_09= pd.concat([M_2017_09, df_return_station_num ],axis=1)
M_2017_10 = df_3[df_3['반납 시간(M)'] == '2017-10'][:158].reset_index()['반납 시간(M)']
df_2017_10= pd.concat([M_2017_10, df_return_station_num ],axis=1)
M_2017_11 = df_3[df_3['반납 시간(M)'] == '2017-11'][:158].reset_index()['반납 시간(M)']
df_2017_11= pd.concat([M_2017_11, df_return_station_num ],axis=1)
M_2017_12 = df_3[df_3['반납 시간(M)'] == '2017-12'][:158].reset_index()['반납 시간(M)']
df_2017_12= pd.concat([M_2017_12, df_return_station_num ],axis=1)
M_2018_01 = df_3[df_3['반납 시간(M)'] == '2018-01'][:158].reset_index()['반납 시간(M)']
df_2018_01= pd.concat([M_2018_01, df_return_station_num ],axis=1)
M_2018_02 = df_3[df_3['반납 시간(M)'] == '2018-02'][:158].reset_index()['반납 시간(M)']
df_2018_02= pd.concat([M_2018_02, df_return_station_num ],axis=1)
M_2018_03 = df_3[df_3['반납 시간(M)'] == '2018-03'][:158].reset_index()['반납 시간(M)']
df_2018_03= pd.concat([M_2018_03, df_return_station_num ],axis=1)
M_2018_04 = df_3[df_3['반납 시간(M)'] == '2018-04'][:158].reset_index()['반납 시간(M)']
df_2018_04= pd.concat([M_2018_04, df_return_station_num ],axis=1)
M_2018_05 = df_3[df_3['반납 시간(M)'] == '2018-05'][:158].reset_index()['반납 시간(M)']
df_2018_05= pd.concat([M_2018_05, df_return_station_num ],axis=1)
M_2018_06 = df_3[df_3['반납 시간(M)'] == '2018-06'][:158].reset_index()['반납 시간(M)']
df_2018_06= pd.concat([M_2018_06, df_return_station_num ],axis=1)
M_2018_07 = df_3[df_3['반납 시간(M)'] == '2018-07'][:158].reset_index()['반납 시간(M)']
df_2018_07= pd.concat([M_2018_07, df_return_station_num ],axis=1)
M_2018_08 = df_3[df_3['반납 시간(M)'] == '2018-08'][:158].reset_index()['반납 시간(M)']
df_2018_08= pd.concat([M_2018_08, df_return_station_num ],axis=1)
M_2018_09 = df_3[df_3['반납 시간(M)'] == '2018-09'][:158].reset_index()['반납 시간(M)']
df_2018_09= pd.concat([M_2018_09, df_return_station_num ],axis=1)
M_2018_10 = df_3[df_3['반납 시간(M)'] == '2018-10'][:158].reset_index()['반납 시간(M)']
df_2018_10= pd.concat([M_2018_10, df_return_station_num ],axis=1)
M_2018_11 = df_3[df_3['반납 시간(M)'] == '2018-11'][:158].reset_index()['반납 시간(M)']
df_2018_11= pd.concat([M_2018_11, df_return_station_num ],axis=1)
M_2018_12 = df_3[df_3['반납 시간(M)'] == '2018-12'][:158].reset_index()['반납 시간(M)']
df_2018_12= pd.concat([M_2018_12, df_return_station_num ],axis=1)
M_2019_01 = df_3[df_3['반납 시간(M)'] == '2019-01'][:158].reset_index()['반납 시간(M)']
df_2019_01= pd.concat([M_2019_01, df_return_station_num ],axis=1)
M_2019_02 = df_3[df_3['반납 시간(M)'] == '2019-02'][:158].reset_index()['반납 시간(M)']
df_2019_02= pd.concat([M_2019_02, df_return_station_num ],axis=1)
M_2019_03 = df_3[df_3['반납 시간(M)'] == '2019-03'][:158].reset_index()['반납 시간(M)']
df_2019_03= pd.concat([M_2019_03, df_return_station_num ],axis=1)
M_2019_04 = df_3[df_3['반납 시간(M)'] == '2019-04'][:158].reset_index()['반납 시간(M)']
df_2019_04= pd.concat([M_2019_04, df_return_station_num ],axis=1)
M_2019_05 = df_3[df_3['반납 시간(M)'] == '2019-05'][:158].reset_index()['반납 시간(M)']
df_2019_05= pd.concat([M_2019_05, df_return_station_num ],axis=1)
M_2019_06 = df_3[df_3['반납 시간(M)'] == '2019-06'][:158].reset_index()['반납 시간(M)']
df_2019_06= pd.concat([M_2019_06, df_return_station_num ],axis=1)
M_2019_07 = df_3[df_3['반납 시간(M)'] == '2019-07'][:158].reset_index()['반납 시간(M)']
df_2019_07= pd.concat([M_2019_07, df_return_station_num ],axis=1)
M_2019_08 = df_3[df_3['반납 시간(M)'] == '2019-08'][:158].reset_index()['반납 시간(M)']
df_2019_08= pd.concat([M_2019_08, df_return_station_num ],axis=1)
M_2019_09 = df_3[df_3['반납 시간(M)'] == '2019-09'][:158].reset_index()['반납 시간(M)']
df_2019_09= pd.concat([M_2019_09, df_return_station_num ],axis=1)
M_2019_10 = df_3[df_3['반납 시간(M)'] == '2019-10'][:158].reset_index()['반납 시간(M)']
df_2019_10= pd.concat([M_2019_10, df_return_station_num ],axis=1)
M_2019_11 = df_3[df_3['반납 시간(M)'] == '2019-11'][:158].reset_index()['반납 시간(M)']
df_2019_11= pd.concat([M_2019_11, df_return_station_num ],axis=1)
M_2019_12 = df_3[df_3['반납 시간(M)'] == '2019-12'][:158].reset_index()['반납 시간(M)']
df_2019_12= pd.concat([M_2019_12, df_return_station_num ],axis=1)
M_2020_01 = df_3[df_3['반납 시간(M)'] == '2020-01'][:158].reset_index()['반납 시간(M)']
df_2020_01= pd.concat([M_2020_01, df_return_station_num ],axis=1)

In [ ]:
M_2020_01 = df_3[df_3['반납 시간(M)'] == '2020-01'][:158].reset_index()['반납 시간(M)']
M_2020_01

0     2020-01
1     2020-01
2     2020-01
3     2020-01
4     2020-01
5     2020-01
6     2020-01
7     2020-01
8     2020-01
9     2020-01
10    2020-01
11    2020-01
12    2020-01
Name: 반납 시간(M), dtype: object

In [ ]:
df_2020_01= pd.concat([M_2020_01, df_return_station_num ],axis=1)
df_2020_01

,반납 시간(M),반납 스테이션 번호
0,2020-01,0
1,2020-01,101
2,2020-01,103
3,2020-01,104
4,2020-01,105
...,...,...
153,NaN,350
154,NaN,351
155,NaN,992
156,NaN,998


In [ ]:
df_all_re = pd.concat([df_2017_01, df_2017_02, df_2017_03, df_2017_04, df_2017_05, df_2017_06, df_2017_07, df_2017_08, df_2017_09, df_2017_10, df_2017_11, df_2017_12, df_2018_01, df_2018_02, df_2018_03, df_2018_04, df_2018_05, df_2018_06, df_2018_07, df_2018_08, df_2018_09, df_2018_10,df_2018_11, df_2018_12, df_2019_01, df_2019_02, df_2019_03, df_2019_04, df_2019_05, df_2019_06, df_2019_07, df_2019_08, df_2019_09, df_2019_10, df_2019_11, df_2019_12, df_2020_01],axis=0)

In [ ]:
df_all_re[-200:]

,반납 시간(M),반납 스테이션 번호
116,2019-12,306
117,2019-12,307
118,2019-12,308
119,2019-12,309
120,2019-12,310
...,...,...
153,NaN,350
154,NaN,351
155,NaN,992
156,NaN,998


In [ ]:
df_all_re.to_csv('월별 반납 스테이션 이용 현황(빈 리스트).csv')
df_all_re = pd.read_csv('월별 반납 스테이션 이용 현황(빈 리스트).csv')

In [ ]:
df_all.to_csv('월별 스테이션 이용 현황(빈 리스트).csv') # 저장
df_alll = pd.read_csv('월별 스테이션 이용 현황(빈 리스트).csv')# 불러오기

In [ ]:
df_alll

,Unnamed: 0,Unnamed: 0.1,대여 시간(M),대여 스테이션 번호
0,0,0,2017-01,101
1,1,1,2017-01,103
2,2,2,2017-01,104
3,3,3,2017-01,105
4,4,4,2017-01,106
...,...,...,...,...
5611,5611,151,2019-12,349
5612,5612,152,2019-12,350
5613,5613,153,2019-12,351
5614,5614,154,2019-12,992
